In [56]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [61]:
house_dict = []

# Filter Out Coming Soon, Auction, Home Type: Apartment, Days On Zillow: 90 Days 
linked = 'https://www.zillow.com/ca/duplex/?searchQueryState={"pagination":{},"usersSearchTerm":"ca","mapBounds":{"west":-152.089851125,"east":-86.523444875,"south":14.485661126627711,"north":55.01789701556767},"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"mapZoom":4,"filterState":{"isPreMarketForeclosure":{"value":false},"isPreMarketPreForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isSingleFamily":{"value":false},"isManufactured":{"value":false},"isCondo":{"value":false},"isMultiFamily":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"doz":{"value":"90"}},"isListVisible":true}'
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKait/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
headers = {'User-Agent': user_agent}
response = requests.get(linked, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
results = soup.find_all('article', class_='list-card')
    
# Loop through returned results
for result in results:
#     print(result)
    # Error handling
    try:
        # Identify and return housing weblink
        house_website = str(result.a.get('href'))
        print(house_website)
        # Identify and return house data
        listing_type = result.find('div', class_="list-card-footer").text;
        price = result.find('div', class_="list-card-price").text
#         print(price);
        full_address = result.find('address', class_="list-card-addr").text
        street = result.find('address', class_="list-card-addr").text.split(",")[0]
        city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
        state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
        try:
            zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
        except IndexError:
            zip_code = 'null'
        bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
        bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
        house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
        # Loop inside each house website and get more house data
        loop_response = requests.get(house_website, headers=headers)
        loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
        
        loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
        loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")
        
        lot = 'null';
        house_type = 'null';
        year_build = 'null';
        parking = 'null';
        hoa = 'null';
        lot_conversion_to_sqft = 'null';
        zillow_price_per_sqft = 'null';

        for content in loop_result2:
            if "Lot" in content.text.split(":")[0]:
                lot = content.text.split(":")[1];
            if "Type" in content.text.split(":")[0]:
                house_type = content.text.split(":")[1];
            if "Year built" in content.text.split(":")[0]:
                year_build = content.text.split(":")[1];
            if "Parking" in content.text.split(":")[0]:
                parking = content.text.split(":")[1];
            if "HOA" in content.text.split(":")[0]:
                hoa = content.text.split(":")[1];
            if "Price/sqft" in content.text.split(":")[0]:
                zillow_price_per_sqft = content.text.split(":")[1];
        try:
            if (str(lot.split(" ")[1]) == "sqft"):
                lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
            elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre")) or (str(lot.split(" ")[1]) == "Acres"):
                lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
            elif (lot == "No Data"):
                lot_conversion_to_sqft = 'null'
            else:
                lot_conversion_to_sqft = 'null'
        except IndexError:
            lot_conversion_to_sqft = 'null'
#         print(loop_soup.findAll('span', class_="Text-c11n-8-18-0__aiai24-0 einFCw"))
        try:
            zestimate = loop_soup.findAll('span', class_="Text-c11n-8-18-0__aiai24-0 einFCw")[1].text
        except:
            zestimate = 'null'
        if (zestimate[0] != '$'):
            zestimate = 'null';
        print(zestimate, 'test')
#                 print('zestimate', zestimate)
#             rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-18-0__aiai24-0 einFCw")[1].text
#             print(rent_zestimate, 'test')

#                 print('zestimate', zestimate)
#             rent_zestimate = 'null'
#             print(zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
        house_obj = {
            "Listing_type": listing_type,
            "Price": price,
            "Full_address": full_address,
            "Street": street,
            "City": city,
            "State": state,
            "Zip_code": zip_code,
            "Bed": bed,
            "Bath": bath,
            "Square_Feet": house_square_feet,
            "Lot": lot,
            "Lot_conversion_to_sqft": lot_conversion_to_sqft,
            "House_type": "Apartment",
            "Year_build": year_build,
            "Parking": parking,
            "HOA": hoa,
            "Zillow_price/sqft": zillow_price_per_sqft,
            "Zestimate": zestimate,
#             "Rent_zestimate": rent_zestimate,
            "House_website": house_website
            }

        #Continue to add data into collection house_dict
        house_dict.append(house_obj)
#         print(house_dict)
        time.sleep(2)
    except AttributeError as e:
        continue

print(house_dict)

https://www.zillow.com/homedetails/762-S-Currier-St-2-Pomona-CA-91766/2076787964_zpid/
$488,969 test
https://www.zillow.com/homedetails/1257-B-St-Walnut-Grove-CA-95690/25951813_zpid/
$450,675 test
https://www.zillow.com/homedetails/1278-A-St-Walnut-Grove-CA-95690/25951810_zpid/
$364,489 test
https://www.zillow.com/homedetails/5778-Bagley-Ave-Twentynine-Palms-CA-92277/17504856_zpid/
$173,742 test
https://www.zillow.com/homedetails/655-N-Towne-Ave-Pomona-CA-91767/21512397_zpid/
$469,566 test
https://www.zillow.com/homedetails/316-Portola-Ave-Portola-CA-96122/2112547061_zpid/
$138,536 test
https://www.zillow.com/homedetails/13686-Us-Highway-50-Kyburz-CA-95720/2077347885_zpid/
$346,502 test
https://www.zillow.com/homedetails/12828-Broyles-Trl-Oregon-House-CA-95962/94705265_zpid/
$246,920 test
https://www.zillow.com/homedetails/13991-River-Rd-122-Walnut-Grove-CA-95690/2077196092_zpid/
$875,007 test
https://www.zillow.com/homedetails/26623-Anderson-St-Boron-CA-93516/18965067_zpid/
$225,001 t

In [62]:
house_data = pd.DataFrame(house_dict)
house_data

,Listing_type,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,Lot,Lot_conversion_to_sqft,House_type,Year_build,Parking,HOA,Zillow_price/sqft,Zestimate,House_website
0,,"$469,900","762 S Currier St #2, Pomona, CA 91766",762 S Currier St #2,Pomona,CA,91766,2,2,"1,022",null,null,Apartment,1948,Covered,null,$460,"$488,969",https://www.zillow.com/homedetails/762-S-Curri...
1,,"$445,000","1257 B St, Walnut Grove, CA 95690",1257 B St,WalnutGrove,CA,95690,4,4,"2,618",null,null,Apartment,1915,0 spaces,$28/mo,$170,"$450,675",https://www.zillow.com/homedetails/1257-B-St-W...
2,,"$359,900","1278 A St, Walnut Grove, CA 95690",1278 A St,WalnutGrove,CA,95690,5,--,"2,100",null,null,Apartment,1950,0 spaces,$24/mo,$171,"$364,489",https://www.zillow.com/homedetails/1278-A-St-W...
3,,"$175,000","5778 Bagley Ave, Twentynine Palms, CA 92277",5778 Bagley Ave,TwentyninePalms,CA,92277,3,--,"1,341",null,null,Apartment,1942,"Garage, Garage - Attached, Covered",null,$130,"$173,742",https://www.zillow.com/homedetails/5778-Bagley...
4,,"$450,000","655 N Towne Ave, Pomona, CA 91767",655 N Towne Ave,Pomona,CA,91767,3,2,"1,398",null,null,Apartment,1914,"Garage, Covered",null,$322,"$469,566",https://www.zillow.com/homedetails/655-N-Towne...
5,,"$142,000","316 Portola Ave, Portola, CA 96122",316 Portola Ave,Portola,CA,96122,4,2,"1,650",null,null,Apartment,1978,0 spaces,null,$86,"$138,536",https://www.zillow.com/homedetails/316-Portola...
6,,"$350,000","13686 Us Highway 50, Kyburz, CA 95720",13686 Us Highway 50,Kyburz,CA,95720,--,--,--,null,null,Apartment,No Data,0 spaces,null,null,"$346,502",https://www.zillow.com/homedetails/13686-Us-Hi...
7,,"$249,000","12828 Broyles Trl, Oregon House, CA 95962",12828 Broyles Trl,OregonHouse,CA,95962,3,--,520,null,null,Apartment,1948,"Carport, Garage - Attached, Covered",null,$479,"$246,920",https://www.zillow.com/homedetails/12828-Broyl...
8,,"$875,000","13991 River Rd #122, Walnut Grove, CA 95690",13991 River Rd #122,WalnutGrove,CA,95690,8,--,"17,975",null,null,Apartment,1991,0 spaces,null,$49,"$875,007",https://www.zillow.com/homedetails/13991-River...
9,,"$225,000","26623 Anderson St, Boron, CA 93516",26623 Anderson St,Boron,CA,93516,6,4,"2,174",null,null,Apartment,1963,"Garage - Attached, Covered",null,$103,"$225,001",https://www.zillow.com/homedetails/26623-Ander...


In [63]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q4-2020_Apartment_page1.csv", index=False, header=True)